# train

In [1]:
import yaml
import os

with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

<ipython-input-1-1499ab5789c7>:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(f)


In [2]:
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
os.makedirs(OUTPUT_ROOT, exist_ok=True)

## load library

In [3]:
# python default library
import os
import shutil
import datetime
import sys
import pickle

# general analysis tool-kit
import numpy as np
import pandas as pd
#from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

# pytorch
import torch
from torch import optim, nn
from torch.utils.tensorboard import SummaryWriter

#from torchsummary import summary

# etc
import yaml
yaml.warnings({'YAMLLoadWarning': False})

# original library
sys.path.append('/home/hiroki/research/dcase2021_task2/src/functions')
import common as com
import pytorch_modeler as modeler
from pytorch_model import ResNet38 as Model
#from pytorch_utils import filtered_load_model
#import models

import librosa
import IPython
import librosa.display

## load config and set logger

In [4]:
log_folder = config['IO_OPTION']['OUTPUT_ROOT']+'/{0}.log'.format(datetime.date.today())
logger = com.setup_logger(log_folder, '00_train.py')

## Setting

In [5]:
# Setting seed
modeler.set_seed(42)

In [6]:
############################################################################
# Setting I/O path
############################################################################
# input dirs
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
dev_dir = INPUT_ROOT + "/dev_data"
add_dev_dir = INPUT_ROOT + "/add_dev_data"
# machine type
machine_types = os.listdir(dev_dir)
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
MODEL_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/models'
TB_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/tb'
OUT_FEATURE_DIR = OUTPUT_ROOT + '/extraction_features'
#os.makedirs(OUTPUT_ROOT, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(TB_DIR, exist_ok=True)
os.makedirs(OUT_FEATURE_DIR, exist_ok=True)
# copy config
shutil.copy('./config.yaml', OUTPUT_ROOT)

'/media/hiroki/HDD1TB/research/dcase2021_task2/output/MahalanobisAD/config.yaml'

In [7]:
torch.cuda.is_available()

True

## make path list

- __train_paths__ ([dict]): Each Machine Type
    - __ToyCar__ ([dict]): Each dataset
        - train ([list]) : paths
        - valid_source ([list]) : paths
        - valid_target ([list]) :paths
    - __ToyTrain__ ([dict]): Each dataset
        - train ([list]) : paths
        - valid_source ([list]) : paths
        - valid_target ([list]) : paths
    - ...

In [8]:
dev_paths = {}

for machine_type in machine_types:
    # dev train
    dev_train_paths = [f"{dev_dir}/{machine_type}/train/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/train")]
    dev_train_paths = sorted(dev_train_paths)
    
    # add_dev train
    #add_dev_paths = [f"{add_dev_dir}/{machine_type}/train/" + file for file in os.listdir(f"{add_dev_dir}/{machine_type}/train")]
    #add_dev_paths = sorted(add_dev_paths)
    
    # dev_source valid
    dev_source_paths = [f"{dev_dir}/{machine_type}/source_test/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/source_test")]
    dev_source_paths = sorted(dev_source_paths)
    
    # dev_target valid
    dev_target_paths = [f"{dev_dir}/{machine_type}/target_test/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/target_test")]
    dev_target_paths = sorted(dev_target_paths)
    
    # bundle
    dev_paths[machine_type] = {}
    dev_paths[machine_type]['train'] = dev_train_paths# + add_train_paths
    dev_paths[machine_type]['valid_source'] = dev_source_paths
    dev_paths[machine_type]['valid_target'] = dev_target_paths

## training

In [9]:
#############################################################################
# run
#############################################################################
def run(machine_type, dev_paths):
    com.tic()
    
    logger.info('TARGET MACHINE_TYPE: {0}'.format(machine_type))
    logger.info('MAKE DATA_LOADER')
    # dev_train_paths
    dataloaders_dict = modeler.make_dataloader(dev_paths, machine_type)
    # parameter setting
    logger.info('FEATURE EXTRACTION')
    net = Model(sample_rate=config['param']['sample_rate'],
                window_size=config['param']['window_size'],
                hop_size=config['param']['hop_size'],
                mel_bins=config['param']['mel_bins'],
                fmin=config['param']['fmin'],
                fmax=config['param']['fmax'])
    pretrained_dict = torch.load(config['IO_OPTION']['PREMODEL_ROOT'])
    net.load_state_dict(pretrained_dict, strict=False)
    output_dicts = modeler.extract_net(net, dataloaders_dict)
    out_path = f'{OUT_FEATURE_DIR}/{machine_type}_features.pkl'
    #pd.to_pickle(output_dicts, out_path)
    logger.info(f'SAVE SUCCESS : {out_path}')
    
    com.toc()

In [10]:
machine_types

['fan', 'gearbox', 'pump', 'slider', 'ToyCar', 'ToyTrain', 'valve']

In [11]:
for machine_type in machine_types:
    run(machine_type, dev_paths)

2021-03-18 16:28:24,131 - 00_train.py - INFO - TARGET MACHINE_TYPE: fan
2021-03-18 16:28:24,131 - 00_train.py - INFO - MAKE DATA_LOADER
2021-03-18 16:28:24,132 - 00_train.py - INFO - FEATURE EXTRACTION
  0%|          | 0/24 [00:00<?, ?it/s]

use: cuda:0
tensor([[0.0000, 0.0000, 1.4522,  ..., 0.0000, 2.4866, 0.4923],
        [0.0000, 0.0000, 1.5812,  ..., 0.0000, 2.7145, 0.5254],
        [0.0000, 0.0000, 1.7496,  ..., 0.0000, 2.9948, 0.5820],
        ...,
        [0.0000, 0.0000, 1.6398,  ..., 0.0000, 2.7994, 0.5495],
        [0.0000, 0.0000, 1.6262,  ..., 0.0000, 2.7924, 0.5402],
        [0.0000, 0.0000, 1.5785,  ..., 0.0000, 2.7091, 0.5245]],
       device='cuda:0')


  4%|▍         | 1/24 [00:01<00:45,  1.98s/it]

tensor([[0.0000, 0.0000, 1.4050,  ..., 0.0000, 2.4118, 0.4765],
        [0.0000, 0.0000, 1.6338,  ..., 0.0000, 2.7875, 0.5535],
        [0.0000, 0.0000, 1.5824,  ..., 0.0000, 2.7182, 0.5255],
        ...,
        [0.0000, 0.0000, 1.8350,  ..., 0.0000, 3.1416, 0.6108],
        [0.0000, 0.0000, 1.5771,  ..., 0.0000, 2.7080, 0.5241],
        [0.0000, 0.0000, 1.7372,  ..., 0.0000, 2.9824, 0.5776]],
       device='cuda:0')


  8%|▊         | 2/24 [00:03<00:37,  1.68s/it]

...

 96%|█████████▌| 23/24 [00:44<00:02,  2.21s/it]

tensor([[0.0000, 0.0000, 1.5796,  ..., 0.0000, 2.6992, 0.5310],
        [0.0000, 0.0000, 1.5298,  ..., 0.0000, 2.6162, 0.5181],
        [0.0000, 0.0000, 1.5223,  ..., 0.0000, 2.6118, 0.5080],
        ...,
        [0.0000, 0.0000, 1.5513,  ..., 0.0000, 2.6588, 0.5200],
        [0.0000, 0.0000, 1.4758,  ..., 0.0000, 2.5295, 0.4974],
        [0.0000, 0.0000, 1.4913,  ..., 0.0000, 2.5532, 0.5011]],
       device='cuda:0')


  0%|          | 0/5 [00:00<?, ?it/s]


file_broken or not exists!! : /media/hiroki/HDD1TB/research/dcase2021_task2/datasets/dev_data/fan/source_test/section_00_source_test_anomaly_0006.wav


TypeError: 'NoneType' object is not subscriptable

In [ ]:
machine_type = 'pump'
input_path = f'{OUT_FEATURE_DIR}/{machine_type}_features.pkl'
ext_data = pd.read_pickle(input_path)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(ext_data['train']['features'], cmap='jet')
plt.colorbar()

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(ext_data['valid_source']['features'], cmap='jet')
plt.colorbar()

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(ext_data['train']['features'], cmap='jet')
plt.colorbar()

history = run('ToyCar')
with open('{}/{}_history.pkl'.format(PKL_DIR, machine_type), 'wb') as file:
    pickle.dump(history , file)

In [ ]:
print(M_means.shape)
plt.imshow(M_means, aspect='auto', cmap='jet')
plt.title(phase)
plt.colorbar()
plt.show()